# 7 PyTorch Tips You Should Know

In [1]:
import time

import torch
from torch import nn

# 1. Create Tensors Directly on the Target Device

In [2]:
start_time = time.time()

for _ in range(100):
  # Creating on the CPU, then transfering to the GPU
  cpu_tensor = torch.ones((1000, 64, 64))
  gpu_tensor = cpu_tensor.cuda()

print('Total time: {:.3f}s'.format(time.time() - start_time))

Total time: 11.250s


In [3]:
start_time = time.time()

for _ in range(100):
  # Creating on GPU directly
  cpu_tensor = torch.ones((1000, 64, 64), device='cuda')

print('Total time: {:.3f}s'.format(time.time() - start_time))

Total time: 0.007s


# 2. Use `Sequential` Layers When Possible

In [4]:
class ExampleModel(nn.Module):
  def __init__(self):
    super().__init__()

    input_size = 2
    output_size = 3
    hidden_size = 16

    self.input_layer = nn.Linear(input_size, hidden_size)
    self.input_activation = nn.ReLU()

    self.mid_layer = nn.Linear(hidden_size, hidden_size)
    self.mid_activation = nn.ReLU()

    self.output_layer = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    z = self.input_layer(x)
    z = self.input_activation(z)
    
    z = self.mid_layer(z)
    z = self.mid_activation(z)
    
    out = self.output_layer(z)

    return out

In [5]:
example_model = ExampleModel()
print(example_model)
print('Output shape:', example_model(torch.ones([100, 2])).shape)

ExampleModel(
  (input_layer): Linear(in_features=2, out_features=16, bias=True)
  (input_activation): ReLU()
  (mid_layer): Linear(in_features=16, out_features=16, bias=True)
  (mid_activation): ReLU()
  (output_layer): Linear(in_features=16, out_features=3, bias=True)
)
Output shape: torch.Size([100, 3])


In [6]:
class ExampleSequentialModel(nn.Module):
  def __init__(self):
    super().__init__()

    input_size = 2
    output_size = 3
    hidden_size = 16

    self.layers = nn.Sequential(
      nn.Linear(input_size, hidden_size),
      nn.ReLU(),
      nn.Linear(hidden_size, hidden_size),
      nn.ReLU(),
      nn.Linear(hidden_size, output_size))

  def forward(self, x):
    out = self.layers(x)
    return out

In [7]:
example_seq_model = ExampleSequentialModel()
print(example_seq_model)
print('Output shape:', example_seq_model(torch.ones([100, 2])).shape)

ExampleSequentialModel(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=3, bias=True)
  )
)
Output shape: torch.Size([100, 3])


# 3. Don't Make Lists of Layers

In [8]:
class BadListModel(nn.Module):
  def __init__(self):
    super().__init__()

    input_size = 2
    output_size = 3
    hidden_size = 16

    self.input_layer = nn.Linear(input_size, hidden_size)
    self.input_activation = nn.ReLU()

    # Fairly common when using residual layers
    self.mid_layers = []
    for _ in range(5):
      self.mid_layers.append(nn.Linear(hidden_size, hidden_size))
      self.mid_layers.append(nn.ReLU())

    self.output_layer = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    z = self.input_layer(x)
    z = self.input_activation(z)
    
    for layer in self.mid_layers:
      z = layer(z)
    
    out = self.output_layer(z)

    return out

In [9]:
bad_list_model = BadListModel()
print('Output shape:', bad_list_model(torch.ones([100, 2])).shape)

Output shape: torch.Size([100, 3])


In [10]:
gpu_input = torch.ones([100, 2], device='cuda')
gpu_bad_list_model = bad_list_model.cuda()
print('Output shape:', bad_list_model(gpu_input).shape)

RuntimeError: ignored

## Better Way to Do This

In [11]:
class CorrectListModel(nn.Module):
  def __init__(self):
    super().__init__()

    input_size = 2
    output_size = 3
    hidden_size = 16

    self.input_layer = nn.Linear(input_size, hidden_size)
    self.input_activation = nn.ReLU()

    # Fairly common when using residual layers
    self.mid_layers = []
    for _ in range(5):
      self.mid_layers.append(nn.Linear(hidden_size, hidden_size))
      self.mid_layers.append(nn.ReLU())
    self.mid_layers = nn.Sequential(*self.mid_layers)

    self.output_layer = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    z = self.input_layer(x)
    z = self.input_activation(z)
    z = self.mid_layers(z)
    out = self.output_layer(z)

    return out

In [12]:
correct_list_model = CorrectListModel()
gpu_input = torch.ones([100, 2], device='cuda')
gpu_correct_list_model = correct_list_model.cuda()
print('Output shape:', correct_list_model(gpu_input).shape)

Output shape: torch.Size([100, 3])


# 4. Make Use of Distributions

In [13]:
# Setup
example_model = ExampleModel()
input_tensor = torch.rand(5, 2)
output = example_model(input_tensor)
print(output)

tensor([[0.1751, 0.3160, 0.0865],
        [0.1616, 0.2483, 0.0599],
        [0.1604, 0.2266, 0.0476],
        [0.1606, 0.2384, 0.0569],
        [0.1193, 0.2597, 0.0577]], grad_fn=<AddmmBackward>)


In [14]:
from torch.distributions import Categorical
from torch.distributions.kl import kl_divergence

In [15]:
dist = Categorical(logits=output)
dist

Categorical(logits: torch.Size([5, 3]))

In [16]:
# Get probabilities
dist.probs

tensor([[0.3261, 0.3755, 0.2984],
        [0.3340, 0.3643, 0.3017],
        [0.3376, 0.3608, 0.3016],
        [0.3353, 0.3624, 0.3023],
        [0.3235, 0.3723, 0.3042]], grad_fn=<SoftmaxBackward>)

In [17]:
# Take samples
dist.sample()

tensor([2, 1, 1, 1, 2])

In [18]:
# Calculate the KL-Divergence
dist_1 = Categorical(logits=output[0])
dist_2 = Categorical(logits=output[1])
kl_divergence(dist_1, dist_2)

tensor(0.0003, grad_fn=<SumBackward1>)

# 5. Use `detach()` On Long-Term Metrics

In [19]:
# Setup
example_model = ExampleModel()
data_batches = [torch.rand((10, 2)) for _ in range(5)]
criterion = nn.MSELoss(reduce='mean')

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


## Bad Example

In [20]:
losses = []

# Training loop
for batch in data_batches:
  output = example_model(batch)

  target = torch.rand((10, 3))
  loss = criterion(output, target)
  losses.append(loss)

  # Optimization happens here

print(losses)

[tensor(0.3803, grad_fn=<MseLossBackward>), tensor(0.2849, grad_fn=<MseLossBackward>), tensor(0.3214, grad_fn=<MseLossBackward>), tensor(0.2970, grad_fn=<MseLossBackward>), tensor(0.2953, grad_fn=<MseLossBackward>)]


## Better Example

In [21]:
losses = []

# Training loop
for batch in data_batches:
  output = example_model(batch)

  target = torch.rand((10, 3))
  loss = criterion(output, target)
  losses.append(loss.item()) # Or `loss.item()`

  # Optimization happens here

print(losses)

[0.42216455936431885, 0.44215714931488037, 0.36152341961860657, 0.42846500873565674, 0.38867220282554626]


# 6. Trick to Delete a Model from GPU

In [22]:
import gc

In [23]:
example_model = ExampleModel().cuda()

del example_model

gc.collect()
# The model will normally stay on the cache until something takes it's place
torch.cuda.empty_cache()

# 7. Call `eval()` Before Testing

In [24]:
example_model = ExampleModel()

# Do training

example_model.eval()

# Do testing

example_model.train()

# Do training again

ExampleModel(
  (input_layer): Linear(in_features=2, out_features=16, bias=True)
  (input_activation): ReLU()
  (mid_layer): Linear(in_features=16, out_features=16, bias=True)
  (mid_activation): ReLU()
  (output_layer): Linear(in_features=16, out_features=3, bias=True)
)

### Affects
  - Dropout
  - Batch Normalization
  - RNNs
  - Lazy Variants

source: https://stackoverflow.com/questions/66534762/which-pytorch-modules-are-affected-by-model-eval-and-model-train